## Search the Best Arbitrage in Bostrom Pools

In [1]:
from subprocess import Popen, PIPE
import requests
import json
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML

POOL_FEE = 0.0015
MIN_ARBITRAGE_REVENUE = 0.02
H_START_AMOUNT = 1_000_000
IBC_COIN_NAMES = \
    {
        'ibc/BA313C4A19DFBF943586C0387E6B11286F9E416B4DD27574E6909CABE0E342FA': 'uatom in bostrom',
        'ibc/13B2C536BB057AC79D5616B8EA1B9540EC1F2170718CAFF6F0083C966FFFED0B': 'uosmo in bostrom',
        'ibc/27394FB092D2ECCD56123C74F36E4C1F926001CEADA9CA97EA622B25F41E5EB2': 'uatom in osmosis',
        'ibc/FE2CD1E6828EC0FAB8AF39BAC45BC25B965BA67CCBC50C13A14BD610B0D1E2C4': 'boot in osmosis'
     }
IBC_COIN_NAMES_INVERT = {v: k for k, v in IBC_COIN_NAMES.items()}
BOOT_RELATED_OSMO_POOLS = (1, 596, 597)
OSMO_POOLS_API_URL = 'https://lcd-osmosis.keplr.app/osmosis/gamm/v1beta1/pools?pagination.limit=750'

#### Get Pool Data from Bostrom

In [2]:
def execute_bash(bash_command: str):
    if len(bash_command.split('"')) == 1:
        _bash_command_list = bash_command.split()
    elif len(bash_command.split('"')) == 2:
        _bash_command_list = \
            bash_command.split('"')[0].split() + \
            [bash_command.split('"')[1]]
    elif len(bash_command.split('"')) > 2:
        _bash_command_list = \
            bash_command.split('"')[0].split() + \
            [bash_command.split('"')[1]] + \
            [item for items in bash_command.split('"')[2:] for item in items.split()]
    else:
        return None, f'Cannot split bash command {bash_command}'
    popen_process = Popen(_bash_command_list, stdout=PIPE)
    return popen_process.communicate(timeout=15)

def get_json_from_bash_query(bash_command: str):
    _res = execute_bash(bash_command)
    if _res[0]:
        return json.loads(_res[0].decode('utf8').replace("'", '"'))
    return

pools_bostrom_json = get_json_from_bash_query('cyber query liquidity pools --node https://rpc.bostrom.cybernode.ai:443 -o json')
pools_bostrom_df = pd.DataFrame(pools_bostrom_json['pools'])
pools_bostrom_df['balances'] = \
    pools_bostrom_df['reserve_account_address'].map(
        lambda address: get_json_from_bash_query(f'cyber query bank balances {address} --node https://rpc.bostrom.cybernode.ai:443 -o json')['balances'])

display(HTML(pools_bostrom_df.to_html(index=False, notebook=True, show_dimensions=False)))

id,type_id,reserve_coin_denoms,reserve_account_address,pool_coin_denom,balances
1,1,"[boot, hydrogen]",bostrom1wrtkzr96362ty7ad0qrwhkpx743xcjrtv7j2cw,pool70D7610CBA8E94B27BAD7806EBD826F5626C486BBF5...,"[{'denom': 'boot', 'amount': '75999147646'}, {'..."
2,1,"[boot, milliampere]",bostrom1y7au5el59mf8mlak53gfvmr75gr2m2jz6lra0s,pool27BBCA67F42ED27DFFB6A450966C7EA206ADAA42BA0...,"[{'denom': 'boot', 'amount': '22272977099'}, {'..."
3,1,"[boot, tocyb]",bostrom1tkpsxklqu74eqsmezcwnc5hmfswrjgn9elcneq,pool5D83035BE0E7AB904379161D3C52FB4C1C392265AC1...,"[{'denom': 'boot', 'amount': '13898478242'}, {'..."
4,1,"[hydrogen, tocyb]",bostrom1kqkwg2eq9fc5r8f0n66fj6ekc5h3kkmqhthkqx,poolB02CE42B202A71419D2F9EB4996B36C52F1B5B60DAF...,"[{'denom': 'hydrogen', 'amount': '33014030265'}..."
5,1,"[hydrogen, milliampere]",bostrom10xu73c3nkcdcf6j6u4j2kzq0cjftfj2nv7xz9y,pool79B9E8E233B61B84EA5AE564AB080FC492B4C953A9D...,"[{'denom': 'hydrogen', 'amount': '230777393227'..."
6,1,"[hydrogen, millivolt]",bostrom1u3u628ue3c5hn7ya4x4cjlk5t2d0unnarrq34k,poolE479A51F998E2979F89DA9AB897ED45A9AFE4E7DE32...,"[{'denom': 'hydrogen', 'amount': '206948089022'..."
7,1,"[hydrogen, ibc/13B2C536BB057AC79D5616B8EA1B9540...",bostrom1np5jg5xq8x7axptrgad8dxd8spg8tavjh5vnaz,pool98692450C039BDD30563475A7699A7805075F592A36...,"[{'denom': 'hydrogen', 'amount': '10000000000'}..."
8,1,"[hydrogen, ibc/BA313C4A19DFBF943586C0387E6B1128...",bostrom1mgv368468aq8fsk2kynh0lpcsgtwa53zzlnhp0,poolDA191D1EBA3F4074C2CAB12777FC388216EED222F57...,"[{'denom': 'hydrogen', 'amount': '19699999999'}..."
9,1,[ibc/13B2C536BB057AC79D5616B8EA1B9540EC1F217071...,bostrom1tyx5u9mgwxzzj6rstsjxmfswpft75s0euh5ppu,pool590D4E176871842968705C246DA60E0A57EA41F9257...,[{'denom': 'ibc/13B2C536BB057AC79D5616B8EA1B954...


#### Get Pool Data from Osmosis

In [3]:
pools_osmosis_json = requests.get(OSMO_POOLS_API_URL).json()['pools']
pools_osmosis_df = pd.DataFrame([item for item in pools_osmosis_json])
pools_osmosis_df['id'] = pools_osmosis_df['id'].astype(int)
pools_osmosis_df['totalWeight'] = pools_osmosis_df['totalWeight'].astype(int)
pools_osmosis_df['balances'] = pools_osmosis_df['poolAssets'].map(lambda x: [item['token'] for item in x])
pools_osmosis_df['denoms_count'] = pools_osmosis_df['poolAssets'].map(lambda x: len(x))
pools_osmosis_df['swapfee'] = pools_osmosis_df['poolParams'].map(lambda x: float(x['swapFee']))
pools_osmosis_df['reserve_coin_denoms'] = pools_osmosis_df['poolAssets'].map(lambda x: [item['token']['denom'] for item in x])
display(HTML(
    pools_osmosis_df[pools_osmosis_df.id.isin(BOOT_RELATED_OSMO_POOLS)]
        .sort_values('totalWeight', ascending=False).to_html(index=False, notebook=True, show_dimensions=False)))

@type,address,id,poolParams,future_pool_governor,totalShares,poolAssets,totalWeight,balances,denoms_count,swapfee,reserve_coin_denoms
/osmosis.gamm.v1beta1.Pool,osmo1mw0ac6rwlp5r8wapwk3zs6g29h8fcscxqakdzw9emk...,1,"{'swapFee': '0.003000000000000000', 'exitFee': ...",24h,"{'denom': 'gamm/pool/1', 'amount': '25180488772...",[{'token': {'denom': 'ibc/27394FB092D2ECCD56123...,1073741824000000,[{'denom': 'ibc/27394FB092D2ECCD56123C74F36E4C1...,2,0.003,[ibc/27394FB092D2ECCD56123C74F36E4C1F926001CEAD...
/osmosis.gamm.v1beta1.Pool,osmo1jtzkz2383cegga8pzq7azm7tp3lcutep95urpvuqxz...,596,"{'swapFee': '0.003000000000000000', 'exitFee': ...",24h,"{'denom': 'gamm/pool/596', 'amount': '114707028...",[{'token': {'denom': 'ibc/27394FB092D2ECCD56123...,1073741824000000,[{'denom': 'ibc/27394FB092D2ECCD56123C74F36E4C1...,2,0.003,[ibc/27394FB092D2ECCD56123C74F36E4C1F926001CEAD...
/osmosis.gamm.v1beta1.Pool,osmo19zg3tz4q5t3x6d2hdmwfkud4d3l3x8r4szr6clvlss...,597,"{'swapFee': '0.002000000000000000', 'exitFee': ...",24h,"{'denom': 'gamm/pool/597', 'amount': '111212167...",[{'token': {'denom': 'ibc/FE2CD1E6828EC0FAB8AF3...,1073741824000000,[{'denom': 'ibc/FE2CD1E6828EC0FAB8AF39BAC45BC25...,2,0.002,[ibc/FE2CD1E6828EC0FAB8AF39BAC45BC25B965BA67CCB...


#### Calculate Prices

In [4]:
pools_df = pools_bostrom_df[['id', 'balances', 'reserve_coin_denoms']].append(
    pools_osmosis_df[(pools_osmosis_df.denoms_count == 2)
    & (pools_osmosis_df.id.isin(BOOT_RELATED_OSMO_POOLS))][['id', 'balances', 'swapfee', 'reserve_coin_denoms']])
coins_list = list(pools_df['reserve_coin_denoms'])
coins_unique_set = set(np.concatenate(coins_list).flat)
price_df = pd.DataFrame(columns=coins_unique_set, index=coins_unique_set)

for index, pool_row in pools_df.iterrows():
    coins_pair = pool_row.reserve_coin_denoms
    balances = {item['denom']: int(item['amount']) for item in pool_row.balances}
    price_df.loc[coins_pair[0], coins_pair[1]] = balances[coins_pair[0]]/balances[coins_pair[1]] * (1 - POOL_FEE)
    price_df.loc[coins_pair[1], coins_pair[0]] = balances[coins_pair[1]]/balances[coins_pair[0]] * (1 - POOL_FEE)
for coin in coins_unique_set:
    price_df.loc[coin, coin] = 1
price_df.rename(columns=IBC_COIN_NAMES, index=IBC_COIN_NAMES, inplace=True)

price_df.loc['uatom in bostrom', 'uatom in osmosis'] = 1
price_df.loc['uatom in osmosis', 'uatom in bostrom'] = 1
price_df.loc['uosmo', 'uosmo in bostrom'] = 1
price_df.loc['uosmo in bostrom', 'uosmo'] = 1
price_df.loc['boot', 'boot in osmosis'] = 1
price_df.loc['boot in osmosis', 'boot'] = 1


coins_unique_set = set(price_df.columns)
display(HTML(price_df.to_html(notebook=True, show_dimensions=False)))

,uosmo,milliampere,millivolt,tocyb,uatom in bostrom,uatom in osmosis,boot,boot in osmosis,hydrogen,uosmo in bostrom
uosmo,1,NaN,NaN,NaN,NaN,5.255613,NaN,0.08297,NaN,1
milliampere,NaN,1,NaN,NaN,NaN,NaN,0.000067,NaN,0.000041,NaN
millivolt,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,0.000009,NaN
tocyb,NaN,NaN,NaN,1,NaN,NaN,0.398259,NaN,0.241098,NaN
uatom in bostrom,NaN,NaN,NaN,NaN,1,1,NaN,NaN,0.014977,0.1997
uatom in osmosis,0.189702,NaN,NaN,NaN,1,1,NaN,0.015682,NaN,NaN
boot,NaN,14827.39657,NaN,2.503401,NaN,NaN,1,1,0.623142,NaN
boot in osmosis,12.016464,NaN,NaN,NaN,NaN,63.577562,1,1,NaN,NaN
hydrogen,NaN,24093.853244,113562.887252,4.135257,66.566667,NaN,1.599961,NaN,1,12.48125
uosmo in bostrom,1,NaN,NaN,NaN,4.9925,NaN,NaN,NaN,0.07988,1


#### Search the Best Arbitrage

In [5]:
result_list = []
for coin_1 in coins_unique_set:
    coin_1_amount = H_START_AMOUNT * price_df.loc[coin_1, 'hydrogen']
    for coin_2 in coins_unique_set.difference({coin_1}):
        coin_2_amount = coin_1_amount * price_df.loc[coin_2, coin_1]
        for coin_3 in coins_unique_set.difference({coin_1, coin_2}):
            coin_3_amount = coin_2_amount * price_df.loc[coin_3, coin_2]
            result = coin_3_amount * price_df.loc[coin_1, coin_3]
            if result > coin_1_amount * (1 + MIN_ARBITRAGE_REVENUE):
                result_list.append(
                        [[coin_1, coin_2, coin_3],
                         round(result/coin_1_amount, 3)])
            for coin_4 in coins_unique_set.difference({coin_1, coin_2, coin_3}):
                coin_4_amount = coin_3_amount * price_df.loc[coin_4, coin_3]
                result = coin_4_amount * price_df.loc[coin_1, coin_4]
                if result > coin_1_amount * (1 + MIN_ARBITRAGE_REVENUE):
                    result_list.append(
                        [[coin_1, coin_2, coin_3, coin_4],
                         round(result/coin_1_amount, 3)])

result_df =\
    pd.DataFrame(
        data=[[' -> '.join(coin for coin in item[0] + [item[0][0]]), item[1]] for item in result_list],
        columns=['arbitrage', 'revenue']).sort_values('revenue', ascending=False)
pd.set_option('display.max_colwidth', None)
display(HTML(result_df.to_html(index=False, notebook=True, show_dimensions=False)))


arbitrage,revenue
uatom in bostrom -> hydrogen -> uosmo in bostrom -> uatom in bostrom,1.062
uosmo in bostrom -> uatom in bostrom -> hydrogen -> uosmo in bostrom,1.062
hydrogen -> uosmo in bostrom -> uatom in bostrom -> hydrogen,1.062
uatom in bostrom -> uatom in osmosis -> uosmo -> uosmo in bostrom -> uatom in bostrom,1.050
uosmo in bostrom -> uatom in bostrom -> uatom in osmosis -> uosmo -> uosmo in bostrom,1.050
tocyb -> hydrogen -> boot -> tocyb,1.026
boot -> tocyb -> hydrogen -> boot,1.026
hydrogen -> boot -> tocyb -> hydrogen,1.026
